In [1]:
#| default_exp payment_clients


# payment clients

> Payment clients are the interfaces and implementations that allow users to pay for L402 offers.

In [2]:
#| export

from abc import ABC, abstractmethod
from datetime import datetime
from typing import List, Dict, Any

from fastcore.utils import *
import fastcore.basics as fc
import os
import json
from l402.payment_providers import *
from l402.utils import *
import httpx
from pydantic import BaseModel


In [3]:
chain = 'base-mainnet'

In [4]:
#| export

class PaymentProvider(ABC):
    """Base class for all payment providers"""
    supported_methods: list[str] = []  # Will be overridden by each provider
    
    @abstractmethod
    def pay(self, *args, **kwargs):
        pass

class PaymentRequest(BaseModel):
    """Represents a payment request to get payment details from a payment provider"""
    offer_id: str
    payment_context_token: str
    payment_method: str
    chain: str = ""
    asset: str = ""


In [5]:
# from l402.utils import *

# run_l402_server(PaymentRequest, port=9000)

## Coinbase Provider

In [6]:
#| export

from cdp import *

class CoinbaseProvider(fc.BasicRepr):
    def __init__(self, wallet: Wallet,
                asset: str = 'usdc'):
        store_attr()
        self.supported_methods=["onchain"]
        self.chain = self.wallet.network_id

    # NOTE: in the fewsats example, we will return an async taskID
    def pay(self,
            amount: float,
            address: str,
            asset: str):
        # TODO, return a generic async task 
        return self.wallet.transfer(amount, asset, address).wait()

In [7]:
c = CoinbaseProvider(wallet=create_test_wallet(fund=False, chain=chain))
c

CoinbaseProvider(wallet=Wallet: (id: 7faea66f-c79e-4569-8ab5-acbf576490d1, network_id: base-mainnet, server_signer_status: None), asset='usdc', supported_methods=['onchain'], chain='base-mainnet')

In [8]:
r = httpx.get('http://localhost:9000/offers')
r.status_code, r.text


(402,
 '{"offers":[{"amount":1,"currency":"USD","description":"Purchase 1 credit for API access","offer_id":"11dee90e-8cb2-47c9-8a89-0f1d76b41c68","payment_methods":["onchain"],"title":"1 Credit Package","type":"one-time"}],"payment_context_token":"0e424aff-241a-4414-872b-f8176b16846a","payment_request_url":"http://localhost:9000/payment_request","version":"0.2.2"}')

In [9]:
o = r.json()
r.status_code, o

(402,
 {'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': '11dee90e-8cb2-47c9-8a89-0f1d76b41c68',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': '0e424aff-241a-4414-872b-f8176b16846a',
  'payment_request_url': 'http://localhost:9000/payment_request',
  'version': '0.2.2'})

In [10]:
data = {
    "offer_id": first(o['offers'])['offer_id'],
    "payment_method": 'onchain',
    "chain": chain,
    "asset": 'usdc',
    "payment_context_token": o['payment_context_token']
    }
r = httpx.post(o['payment_request_url'], json=data)
r.status_code, r.text

(200,
 '{"expires_at":"2025-01-25T02:55:04.077807+00:00","offer_id":"11dee90e-8cb2-47c9-8a89-0f1d76b41c68","payment_request":{"address":"0xff2BF23F87809E3E3456C2b4a00a8D8e78957052","chain":"base-mainnet","asset":"usdc"},"version":"0.2.2"}')

In [11]:
#| export

def get_payment_request(payment_request_url: str,
                        payment_context_token: str,
                        offer_id: str, 
                        payment_method: str, 
                        chain: str = "", 
                        asset: str = ""):
    data = {
        "offer_id": offer_id,
        "payment_method": payment_method,
        "chain": chain,
        "asset": asset,
        "payment_context_token": payment_context_token
    }
    r = httpx.post(payment_request_url, json=data)
    r.raise_for_status()
    return r.json()


In [12]:
r = get_payment_request(o['payment_request_url'], o['payment_context_token'], o['offers'][0]['offer_id'], 'onchain', chain, 'usdc')
r


{'expires_at': '2025-01-25T02:55:04.596581+00:00',
 'offer_id': '11dee90e-8cb2-47c9-8a89-0f1d76b41c68',
 'payment_request': {'address': '0x58e4daaC484adA3D4ff9C78fCaAb072CDEeD9a8e',
  'chain': 'base-mainnet',
  'asset': 'usdc'},
 'version': '0.2.2'}

In [13]:
#| export

class Client(fc.BasicRepr):
    def __init__(self, lightning_provider = None, 
                 credit_card_provider = None, 
                 onchain_provider = None):
        store_attr()
        self.lightning_provider = lightning_provider
        self.credit_card_provider = credit_card_provider
        self.onchain_provider = onchain_provider


    def pay(self, ofr: dict): # ofr is the l402 offers response dictionary
        "Pay for an offer"
        # this actually does 3 things
        # 1. Selects offer
        # 2. Gets payment request details
        # 3. Uses user-provided payment method
        
        ofr = L402Response(**ofr)
        if len(ofr.offers) != 1: raise ValueError("Only one offer is supported")
        o = first(ofr.offers)

        if 'onchain' in o.payment_methods and self.onchain_provider:
            r = get_payment_request(ofr.payment_request_url, ofr.payment_context_token, o.offer_id, 'onchain', self.onchain_provider.chain, self.onchain_provider.asset)

            return self.onchain_provider.pay(o.amount, r['payment_request']['address'], r['payment_request']['asset'])
            
        # elif 'lightning' in o.payment_methods and self.lightning_provider:
        # elif 'credit_card' in o.payment_methods and self.credit_card_provider:
        else:
            raise ValueError(f"No payment provider available for {ofr.offers[0].payment_methods}")


In [14]:
w = create_test_wallet(fund=False, chain=chain)
c = Client(onchain_provider=CoinbaseProvider(wallet=w, asset='usdc'))
try:
    c.pay(o)
except Exception as e:
    print(e)


Insufficient funds: have 0, need 1.


## Fewsats Client

In [15]:
#| export

# class PaymentStatus:
#     PENDING = "pending"
#     COMPLETED = "completed"
#     FAILED = "failed"
#     EXPIRED = "expired"

#| export

class Fewsats:
    def __init__(self, api_key: str = None, base_url: str = "https://hub-5n97k.ondigitalocean.app"):
        self.api_key = api_key or os.environ.get("FEWSATS_API_KEY")
        if not self.api_key:
            raise ValueError("API key not provided and FEWSATS_API_KEY environment variable is not set")
        self.base_url = base_url
        self.client = httpx.Client()
        self.client.headers.update({"Authorization": f"Token {self.api_key}"})


    def _request(self, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
        url = f"{self.base_url}/{endpoint}"
        response = self.client.request(method, url, **kwargs)
        response.raise_for_status()
        return response.json()

    def get_payment_methods(self) -> List[Dict[str, Any]]:
        """Retrieve the user's payment methods.
        
        Returns:
            List[Dict[str, Any]]: A list of payment methods associated with the user's account.
        """
        return self._request("GET", "v0/stripe/payment-methods")


    def pay(self, ofr: dict):
        data = {
            "payment_request_url": ofr["payment_request_url"],
            "payment_context_token": ofr["payment_context_token"],
            "payment_method": "onchain",
            "offer": first(ofr["offers"])
        }
        print(data)
        print(self.base_url)
        print(self.api_key)
        return self._request("POST", "v0/l402/purchases/from-offer", json=data)

In [16]:
f = Fewsats(base_url='http://localhost:8000', api_key='yQbJPudW-nqiTyx880t8G83oLGyoR-RlnrLDmEe0D58')
f.get_payment_methods()


[{'id': 1,
  'last4': '4242',
  'brand': 'visa',
  'exp_month': 12,
  'exp_year': 2034,
  'is_default': False},
 {'id': 4,
  'last4': '4242',
  'brand': 'Visa',
  'exp_month': 12,
  'exp_year': 2034,
  'is_default': True}]

In [17]:
try:
    f.pay(o)
except Exception as e:
    print(e)

{'payment_request_url': 'http://localhost:9000/payment_request', 'payment_context_token': '0e424aff-241a-4414-872b-f8176b16846a', 'payment_method': 'onchain', 'offer': {'amount': 1, 'currency': 'USD', 'description': 'Purchase 1 credit for API access', 'offer_id': '11dee90e-8cb2-47c9-8a89-0f1d76b41c68', 'payment_methods': ['onchain'], 'title': '1 Credit Package', 'type': 'one-time'}}
http://localhost:8000
yQbJPudW-nqiTyx880t8G83oLGyoR-RlnrLDmEe0D58
Server error '500 Internal Server Error' for url 'http://localhost:8000/v0/l402/purchases/from-offer'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500


In [18]:
#| export

@patch
def _pay_onchain(self: Fewsats, address: str,
                    amount: str,
                    chain: str = '',
                    asset: str = ''):
    data = {
        "address": address,
        "amount": str(amount),
        "chain": chain,
        "asset": asset,
    }
    print(data)
    return self._request("POST", "v0/l402/purchases/onchain", json=data)


In [19]:
try:
    f._pay_onchain(address=r['payment_request']['address'], amount="0.000001", chain='base-mainnet', asset='usdc')
except Exception as e:
    print(e)

{'address': '0x58e4daaC484adA3D4ff9C78fCaAb072CDEeD9a8e', 'amount': '0.000001', 'chain': 'base-mainnet', 'asset': 'usdc'}


In [20]:
o, r

({'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': '11dee90e-8cb2-47c9-8a89-0f1d76b41c68',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': '0e424aff-241a-4414-872b-f8176b16846a',
  'payment_request_url': 'http://localhost:9000/payment_request',
  'version': '0.2.2'},
 {'expires_at': '2025-01-25T02:55:04.596581+00:00',
  'offer_id': '11dee90e-8cb2-47c9-8a89-0f1d76b41c68',
  'payment_request': {'address': '0x58e4daaC484adA3D4ff9C78fCaAb072CDEeD9a8e',
   'chain': 'base-mainnet',
   'asset': 'usdc'},
  'version': '0.2.2'})